<a href="https://colab.research.google.com/github/eppuoskari/wine-dataset/blob/main/wine_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/WineDataset.csv")
df

,Title,Description,Price,Capacity,Grape,Secondary Grape Varieties,Closure,Country,Unit,Characteristics,Per bottle / case / each,Type,ABV,Region,Style,Vintage,Appellation
0,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,£9.99 per bottle,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",per bottle,Red,ABV 14.00%,NaN,Rich & Juicy,NV,NaN
1,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,£15.99 per bottle,75CL,Chardonnay,NaN,Natural Cork,USA,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",per bottle,White,ABV 13.50%,California,Rich & Toasty,2021,Napa Valley
2,"Oyster Bay Sauvignon Blanc 2022, Marlborough",Oyster Bay has been an award-winning gold-stan...,£12.49 per bottle,75CL,Sauvignon Blanc,NaN,Screwcap,New Zealand,9.8,"Tropical Fruit, Gooseberry, Grapefruit, Grass,...",per bottle,White,ABV 13.00%,Marlborough,Crisp & Zesty,2022,NaN
3,Louis Latour Mâcon-Lugny 2021/22,We’ve sold this wine for thirty years – and fo...,£17.99 per bottle,75CL,Chardonnay,NaN,Natural Cork,France,10.1,"Peach, Apricot, Floral, Lemon",per bottle,White,ABV 13.50%,Burgundy,Ripe & Rounded,2022,Macon
4,Bread & Butter 'Winemaker's Selection' Pinot N...,Bread & Butter is that thing that you can coun...,£15.99 per bottle,75CL,Pinot Noir,NaN,Natural Cork,USA,10.1,"Smoke, Black Cherry, Cedar, Raspberry, Red Fruit",per bottle,Red,ABV 13.50%,California,Smooth & Mellow,2021,Napa Valley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,"Vouvray Sec 'Expresion de Silex' 2020/21, Loire",Although Denis Meunier is only in his late 20s...,£11.99 per bottle,75CL,Chenin Blanc,NaN,Natural Cork,France,9.4,"Quince, Green Apple, Lemon, Pear",per bottle,White,ABV 12.50%,Loire,Aromatic & Floral,2021,Vouvray
1286,"Waimea Estates Pinot Noir Rosé 2021, Nelson",Waimea’s vineyards are less than 1km from the ...,£13.99 per bottle,75CL,Pinot Noir,NaN,Screwcap,New Zealand,9.8,"Floral, Raspberry, Red Cherry, Strawberry",per bottle,Rosé,ABV 13.00%,Nelson,Delicate & Dry,2021,NaN
1287,Wakefield 'Visionary' Cabernet Sauvignon 2010,Made from the very best fruit nourished by 40-...,£85.00 per bottle,75CL,Cabernet Sauvignon,NaN,Screwcap,Australia,10.1,"Vanilla, Blackberry, Blackcurrant, Cedar, Euca...",per bottle,Red,ABV 13.50%,South Australia,Savoury & Full Bodied,2010,Clare Valley
1288,Yalumba 'The Menzies' Cabernet Sauvignon 2015/...,"In 1961, Sir Robert Menzies said that Yalumba'...",£37.99 per bottle,75CL,Cabernet Sauvignon,NaN,Natural Cork,Australia,10.9,"Black Plum, Blackcurrant, Vanilla",per bottle,Red,ABV 14.50%,South Australia,Savoury & Full Bodied,2016,Coonawarra


In [ ]:
#Column for Wine_ID

df['Wine_ID'] = range(1, len(df) + 1)

In [ ]:
# Country

df["Country"] = df["Country"].str.strip()
df['Country'] = df['Country'].replace({'USA': 'United States'})
df = df.dropna(subset=["Country"])


In [ ]:
# Style

df['Style'] = df['Style'].astype(str).str.strip().str.split(' & ')
df = df.explode('Style').reset_index(drop=True)
df['Style'] = df['Style'].str.strip()

/tmp/ipython-input-3568096378.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Style'] = df['Style'].astype(str).str.strip().str.split(' & ')


In [ ]:
# Normalizing the Vintage column

df = df.rename(columns={"Vintage": "Harvest year"})

df["Harvest year"] = df["Harvest year"].astype(str).str.split("/")

df = df.explode("Harvest year", ignore_index=True)

df["Harvest year"] = (
    df["Harvest year"]
    .str.strip()
    .replace({"nan": np.nan, "NV": np.nan})
)

# AAAA format
def format_year(val):
    if pd.isna(val):
        return np.nan
    if val.isdigit():
        val_int = int(val)
        if val_int < 100:
            if val_int <= 25:
                return 2000 + val_int
            else:
                return 1900 + val_int
        return val_int
    return np.nan

df["Harvest year"] = df["Harvest year"].apply(format_year).astype("Int64")

In [ ]:
 # Price

# Cleaning the Price column

# This pattern removes anything that’s not part of a number
# 	r"[^\d.]" is a regular expression:
#	\d means “any digit” (0–9).
#	. means a literal decimal point.
#	[^...] means “anything NOT in the brackets.”
#	So "[^\d.]" means “any character that is not a digit or a decimal point.”
#	"" means we replace it with nothing (effectively deleting it).
#	regex=True tells Pandas to treat the first argument as a regular expression.

df["Price"] = df["Price"].replace(r"[^\d.]", "", regex=True).astype(float)

In [ ]:
# Normalizing the Type column

# Extract unique types
types = df['Type'].unique()

# Create a DataFrame with type_id and type_name
type_df = pd.DataFrame({
    'type_id': range(1, len(types) + 1),
    'type_name': types
})

# This is 'type' table, save as csv, for now commented out
# type_df
# type_df.to_csv("type.csv", index=False)

# Create a dictionary for mapping type_id and type_name
type_dict = dict(zip(type_df['type_name'], type_df['type_id']))

# Modify 'Type' column by mapping type_id
df['Type'] = df['Type'].map(type_dict)
df.head()

,Title,Description,Price,Capacity,Grape,Secondary Grape Varieties,Closure,Country,Unit,Characteristics,Per bottle / case / each,Type,ABV,Region,Style,Harvest year,Appellation,Wine_ID
0,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,9.99,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",per bottle,1,ABV 14.00%,NaN,Rich,<NA>,NaN,1
1,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,9.99,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",per bottle,1,ABV 14.00%,NaN,Juicy,<NA>,NaN,1
2,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,15.99,75CL,Chardonnay,NaN,Natural Cork,United States,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",per bottle,2,ABV 13.50%,California,Rich,2021,Napa Valley,2
3,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,15.99,75CL,Chardonnay,NaN,Natural Cork,United States,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",per bottle,2,ABV 13.50%,California,Toasty,2021,Napa Valley,2
4,"Oyster Bay Sauvignon Blanc 2022, Marlborough",Oyster Bay has been an award-winning gold-stan...,12.49,75CL,Sauvignon Blanc,NaN,Screwcap,New Zealand,9.8,"Tropical Fruit, Gooseberry, Grapefruit, Grass,...",per bottle,2,ABV 13.00%,Marlborough,Crisp,2022,NaN,3


In [ ]:
# Normalizing the Capacity column

# Write the function for conversion

def convert_to_liters(value):
    value = value.strip().upper()

    # Special case for "Our"
    if value == "OUR":
        return 4.5

    # Convert common formats
    if value.endswith("CL"):
        return float(value.replace("CL", "")) / 100
    elif value.endswith("ML"):
        return float(value.replace("ML", "")) / 1000
    elif value.endswith("LTR"):
        return float(value.replace("LTR", ""))
    elif value.endswith("LITRE"):
        return float(value.replace("LITRE", ""))
    elif value.endswith("L"):
        return float(value.replace("L", ""))
    else:
        return None  # Unknown format

# Apply conversion
df['Capacity_Liters'] = df['Capacity'].apply(convert_to_liters)

# Check results
df[['Capacity', 'Capacity_Liters']].head(15)

,Capacity,Capacity_Liters
0,75CL,0.75
1,75CL,0.75
2,75CL,0.75
3,75CL,0.75
4,75CL,0.75
5,75CL,0.75
6,75CL,0.75
7,75CL,0.75
8,75CL,0.75
9,75CL,0.75


In [ ]:
# Split the 'Secondary Grape Varieties' column into 4 columns
split_grapes = df['Secondary Grape Varieties'].str.split(', ', expand=True, n=3)

# Create exactly 4 column names
new_column_names = [f'Grape_{i+1}' for i in range(4)]
split_grapes.columns = new_column_names

# Strip
for col in new_column_names:
    split_grapes[col] = split_grapes[col].str.strip()

# Concatenate the new columns to the original DataFrame
df = pd.concat([df, split_grapes], axis=1)

# Drop the original column if desired
#df = df.drop(columns=['Secondary Grape Varieties'])


print(df.head())

                                               Title  \
0                                 The Guv'nor, Spain   
1                                 The Guv'nor, Spain   
2  Bread & Butter 'Winemaker's Selection' Chardon...   
3  Bread & Butter 'Winemaker's Selection' Chardon...   
4       Oyster Bay Sauvignon Blanc 2022, Marlborough   

                                         Description  Price Capacity  \
0  We asked some of our most prized winemakers wo...   9.99     75CL   
1  We asked some of our most prized winemakers wo...   9.99     75CL   
2  This really does what it says on the tin. It’s...  15.99     75CL   
3  This really does what it says on the tin. It’s...  15.99     75CL   
4  Oyster Bay has been an award-winning gold-stan...  12.49     75CL   

             Grape Secondary Grape Varieties       Closure        Country  \
0      Tempranillo                       NaN  Natural Cork          Spain   
1      Tempranillo                       NaN  Natural Cork          Spain   

In [ ]:
# ----- PRIMARY GRAPE & CLOSURE-----

# ----- Normalize Grape -----
unique_grapes = df['Grape'].unique()
grape_lookup = pd.DataFrame({
    'grape_id': range(1, len(unique_grapes) + 1),
    'grape_name': unique_grapes
})
grape_dict = dict(zip(grape_lookup['grape_name'], grape_lookup['grape_id']))
df['grape_id'] = df['Grape'].map(grape_dict)


# ----- Normalize Closure -----
closures = df['Closure'].unique()
closure_lookup = pd.DataFrame({
    'closure_id': range(1, len(closures) + 1),
    'closure_name': closures
})
closure_dict = dict(zip(closure_lookup['closure_name'], closure_lookup['closure_id']))
df['closure_id'] = df['Closure'].map(closure_dict)

# ----- Final normalized DataFrames -----

normalized_grape = df[['Grape', 'grape_id']]
normalized_closure = df[['Closure', 'closure_id']]
normalized_gr_cl = df[['Grape', 'grape_id', 'Closure', 'closure_id']]

# Merge df with grape_lookup
df_grape = df.merge(grape_lookup, left_on='Grape', right_on='grape_name')

# Merge df with closure_lookup
df_closure = df.merge(closure_lookup, left_on='Closure', right_on='closure_name')


In [ ]:
# REGION AND APPELLATION CLEANING
import re
import numpy as np
import pandas as pd

source_col = 'Region (Maria)' if 'Region (Maria)' in df.columns else 'Region'

def clean_region(val: object) -> object:
    """
    Normalizes region names:
      - trims whitespace and collapses multiple spaces
      - converts 'nan'/'NA' to real NaN
      - title-cases but keeps small words (and, of, de, la, etc.) in lowercase
      - uppercases common wine acronyms (DOCG, DOC, AOC, AOP, IGT, IGP, etc.)
      - normalizes ' & ' to ' and '
    """
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    if s == "" or s.lower() in {"nan", "n/a", "na", "none"}:
        return np.nan

    s = re.sub(r"\s+", " ", s)             # single spaces
    s = s.replace(" & ", " and ")          # Connector

    # title case then keep small words lowercase
    t = s.title()
    small = {"and","of","the","de","del","la","los","las","da","do","dos","das","di","el","le","les"}
    parts = []
    for i, token in enumerate(t.split(" ")):
        if "-" in token:  # keep hyphenated compounds neat
            subs = token.split("-")
            subs = [sub if j==0 or sub.lower() not in small else sub.lower() for j, sub in enumerate(subs)]
            token = "-".join(subs)
        elif i > 0 and token.lower() in small:
            token = token.lower()
        parts.append(token)
    t = " ".join(parts)

    # uppercase key wine acronyms anywhere they appear
    for ac in ["DOCG","DOC","AOC","AOP","DOP","IGT","IGP","DO","VDP","PDO","PGI"]:
        t = re.sub(rf"\b{ac}\b", ac, t, flags=re.IGNORECASE)

    return t

df['Region (Maria)'] = df[source_col].apply(clean_region)

In [ ]:
# ID MAPPING
unique_regions = (
    df['Region (Maria)']
    .dropna()
    .drop_duplicates()
    .sort_values()
)

region_lookup = pd.DataFrame({
    'region_id': range(1, len(unique_regions) + 1),
    'region_name': unique_regions.values
})

region_dict = dict(zip(region_lookup['region_name'], region_lookup['region_id']))
df['region_id'] = df['Region (Maria)'].map(region_dict).astype('Int64')

In [ ]:
# Drop Appellation column
appellation_cols = [c for c in df.columns if c.strip().lower() == 'appellation']
df.drop(columns=appellation_cols, inplace=True, errors='ignore')

In [ ]:
df.rename(columns={'Region (Maria)': 'Region'}, inplace=True)

In [ ]:
df

,Title,Description,Price,Capacity,Grape,Secondary Grape Varieties,Closure,Country,Unit,Characteristics,...,Wine_ID,Capacity_Liters,Grape_1,Grape_2,Grape_3,Grape_4,grape_id,closure_id,Region,region_id
0,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,9.99,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",...,1,0.75,NaN,NaN,NaN,NaN,1,1,NaN,<NA>
1,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,9.99,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",...,1,0.75,NaN,NaN,NaN,NaN,1,1,NaN,<NA>
2,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,15.99,75CL,Chardonnay,NaN,Natural Cork,United States,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",...,2,0.75,NaN,NaN,NaN,NaN,2,1,California,14
3,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,15.99,75CL,Chardonnay,NaN,Natural Cork,United States,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",...,2,0.75,NaN,NaN,NaN,NaN,2,1,California,14
4,"Oyster Bay Sauvignon Blanc 2022, Marlborough",Oyster Bay has been an award-winning gold-stan...,12.49,75CL,Sauvignon Blanc,NaN,Screwcap,New Zealand,9.8,"Tropical Fruit, Gooseberry, Grapefruit, Grass,...",...,3,0.75,NaN,NaN,NaN,NaN,3,2,Marlborough,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,Wakefield 'Visionary' Cabernet Sauvignon 2010,Made from the very best fruit nourished by 40-...,85.00,75CL,Cabernet Sauvignon,NaN,Screwcap,Australia,10.1,"Vanilla, Blackberry, Blackcurrant, Cedar, Euca...",...,1288,0.75,NaN,NaN,NaN,NaN,13,2,South Australia,74
2515,Yalumba 'The Menzies' Cabernet Sauvignon 2015/...,"In 1961, Sir Robert Menzies said that Yalumba'...",37.99,75CL,Cabernet Sauvignon,NaN,Natural Cork,Australia,10.9,"Black Plum, Blackcurrant, Vanilla",...,1289,0.75,NaN,NaN,NaN,NaN,13,1,South Australia,74
2516,Yalumba 'The Menzies' Cabernet Sauvignon 2015/...,"In 1961, Sir Robert Menzies said that Yalumba'...",37.99,75CL,Cabernet Sauvignon,NaN,Natural Cork,Australia,10.9,"Black Plum, Blackcurrant, Vanilla",...,1289,0.75,NaN,NaN,NaN,NaN,13,1,South Australia,74
2517,"Yalumba 'Virgilius' Viognier 2017, Eden Valley",Yalumba's Virgilius is often regarded as Austr...,34.99,75CL,Viognier,NaN,Screwcap,Australia,10.1,"Peach, Apricot, Floral, Honeysuckle",...,1290,0.75,NaN,NaN,NaN,NaN,15,2,NaN,<NA>


ValueError: DataFrame columns must be unique. Duplicate columns: ['Region']

In [ ]:
import pandas as pd

df.to_csv("wine_cleaned.csv", index=False, encoding="utf-8-sig")

print("✅ Saved to 'wine_cleaned.csv'")

✅ Saved to 'wine_cleaned.csv'
